# **# Import Python Packages**



In [19]:
# 1. IMPORT ALL PACKAGES
import pandas as pd
from sklearn.metrics import confusion_matrix
import math
from collections import Counter
import seaborn as sns
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn import svm
from sklearn.tree import export_graphviz
# import pydot
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import multilabel_confusion_matrix
from statistics import mean 


import joblib

In [3]:
#CONNECT G COLAB N G DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#LOAD TESTING AND TRAINING DATASET
# header 0 means the first row is name of the coloumn 
df_train = ("/content/sample_data/order_reviews_train_data.csv")
df_test = ("/content/sample_data/order_reviews_test_data.csv")

df_train = pd.read_csv(df_train, sep=';')
df_test = pd.read_csv(df_test, sep=';')

# Drop rows if missing values
df_train = df_train.dropna(subset=['review_comment_message']) 
df_test = df_test.dropna(subset=['review_comment_message']) 

# Check all classes/labels in training data
all_label = set(df_train['score'].tolist())


# Data distribution for each review_score
dst_train = Counter(df_train['score'])
#print(dst_train)


In [ ]:
#REMOVE UNECESSARY COLOUMN
df_test = df_test.drop(df_test.columns[[0, 1, 3, 5, 6]], axis=1)  
df_test
df_test.dropna()

# LABELING BASE ON SCORE

*   4 and 5 are positive
*   3 is neutral
*   1 and 2 are netagive






In [ ]:
#Create Label base on review_score
Label = []
for index, row in df_test.iterrows():
  if row["review_score"] == 4:
    Label.append("Positive")
  elif row["review_score"] ==5:
    Label.append("Positive")
  elif row["review_score"] == 3:
    Label.append("Neutral")
  else: 
    Label.append("Negatif")

df_test["Label"]= Label
df_test['Label'].value_counts()

In [ ]:
#Create Label base on label
score = []
for index, row in df_test.iterrows():
  if row["Label"] == 'Positive':
    score.append(3)
  elif row["Label"] =='Neutral':
    score.append(2)
  else: 
    score.append(1)

df_test["score"]= score
df_test = df_test.drop(df_test.columns[[0]], axis=1) 


In [5]:
#Review The Data
df_test

,review_comment_message,Label,score
0,"NÃ£o recebi o produto, solicito reembolso!",Negatif,1
1,Apesar de chegar no Ãºltimo dia do prazo estab...,Negatif,1
2,"A caixa dÃ¡ a impressÃ£o de produto usado, a q...",Negatif,1
3,Meu profundo veio com defeito! JÃ¡ solicitei t...,Negatif,1
4,"No rastreio fala que foi entregue, porÃ©m nÃ£o...",Negatif,1
...,...,...,...
4850,Obrigada! Ã“timo procedimento!!,Positive,3
4851,recebi,Positive,3
4852,"Chegou muito antes do prazo, produto conforme ...",Positive,3
4853,O pedido chegou antes do previsto !,Positive,3


In [11]:
#SAVE CORPUS DATA AS A FILE
corpus_data = df_train.append(df_test) # from testing and training data
corpus = corpus_data['review_comment_message'].to_numpy()

# 1-gram
model_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 1))
model_vectorizer.fit(corpus)

# Printing the identified Unique words along with their indices
print("Vocabulary: ", model_vectorizer.vocabulary_)

# saving pkl
joblib.dump(model_vectorizer, "/content/sample_data/vectorizer_review.pkl")

Vocabulary:  {'nã': 3397, 'recebi': 4190, 'produto': 3990, 'solicito': 4634, 'reembolso': 4249, 'apesar': 453, 'de': 1444, 'chegar': 995, 'no': 3344, 'ãºltimo': 5246, 'dia': 1647, 'do': 1726, 'prazo': 3884, 'estabelecido': 2061, 'demorou': 1493, 'quase': 4092, 'um': 4995, 'mãªs': 3269, 'para': 3562, 'vir': 5152, 'uma': 4996, 'entrega': 1923, 'tã': 4979, 'simples': 4585, 'isso': 2792, 'que': 4094, 'dã': 1767, 'terceirizar': 4798, 'tudo': 4972, 'ser': 4546, 'responsã': 4337, 'vel': 5075, 'direta': 1682, 'pela': 3648, 'comercializaã': 1117, 'caixa': 845, 'impressã': 2650, 'usado': 5019, 'qualidade': 4080, 'deixa': 1467, 'desejar': 1563, 'como': 1122, 'foi': 2349, 'minha': 3168, 'neta': 3326, 'passou': 3619, 'mas': 3086, 'guardei': 2548, 'nota': 3362, 'se': 4499, 'der': 1508, 'problema': 3964, 'vou': 5193, 'reclamar': 4215, 'pois': 3803, 'achei': 199, 'caro': 907, 'aparencia': 437, 'meu': 3149, 'profundo': 3997, 'veio': 5072, 'com': 1103, 'defeito': 1455, 'jã': 2840, 'solicitei': 4633, 'tr

['/content/sample_data/vectorizer_review.pkl']

In [12]:
#FEATURE EXTRACTION TECHNIQUES
# load object vectorizer (vocabulary)
vectorizer = joblib.load("/content/sample_data/vectorizer_review.pkl")

print("Vocabulary: ", vectorizer.vocabulary_)


Vocabulary:  {'nã': 3397, 'recebi': 4190, 'produto': 3990, 'solicito': 4634, 'reembolso': 4249, 'apesar': 453, 'de': 1444, 'chegar': 995, 'no': 3344, 'ãºltimo': 5246, 'dia': 1647, 'do': 1726, 'prazo': 3884, 'estabelecido': 2061, 'demorou': 1493, 'quase': 4092, 'um': 4995, 'mãªs': 3269, 'para': 3562, 'vir': 5152, 'uma': 4996, 'entrega': 1923, 'tã': 4979, 'simples': 4585, 'isso': 2792, 'que': 4094, 'dã': 1767, 'terceirizar': 4798, 'tudo': 4972, 'ser': 4546, 'responsã': 4337, 'vel': 5075, 'direta': 1682, 'pela': 3648, 'comercializaã': 1117, 'caixa': 845, 'impressã': 2650, 'usado': 5019, 'qualidade': 4080, 'deixa': 1467, 'desejar': 1563, 'como': 1122, 'foi': 2349, 'minha': 3168, 'neta': 3326, 'passou': 3619, 'mas': 3086, 'guardei': 2548, 'nota': 3362, 'se': 4499, 'der': 1508, 'problema': 3964, 'vou': 5193, 'reclamar': 4215, 'pois': 3803, 'achei': 199, 'caro': 907, 'aparencia': 437, 'meu': 3149, 'profundo': 3997, 'veio': 5072, 'com': 1103, 'defeito': 1455, 'jã': 2840, 'solicitei': 4633, 'tr

In [14]:
# Generate feature [test and train] from dictionary
# Testing data
test_data = vectorizer.transform(df_test['review_comment_message']).toarray()
feature_test = pd.DataFrame(data = test_data, columns = vectorizer.get_feature_names_out())

# training data
train_data = vectorizer.transform(df_train['review_comment_message']).toarray()
feature_train = pd.DataFrame(data = train_data, columns = vectorizer.get_feature_names_out())

feature_test

,00,001689,00x2,01,02,03,04,05,05ml,06,...,ãºnica,ãºnico,ãºteis,ãºtil,ãšltima,ãšnica,ãšnico,ãƒo,ðÿ,šðÿ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#ADD CLASS TO TESTING AND TRAINING DATA
# ['POSITIF','NEGATIF', 'NETRAL'], [3, 1, 2]

# train
class_train = df_train['score'];
feature_train['score'] = class_train

# test
class_test = df_test['score'];
feature_test['score'] = class_test

# Drop rows if missing values
feature_test = feature_test.dropna()
feature_train = feature_train.dropna() 
feature_test

,00,001689,00x2,01,02,03,04,05,05ml,06,...,ãºnico,ãºteis,ãºtil,ãšltima,ãšnica,ãšnico,ãƒo,ðÿ,šðÿ,score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [16]:
#DEFINE X AND Y VARIABLES
# Testing data
x_test = feature_test.iloc[:, :-1]
y_test = feature_test['score'].tolist()

# Training data
x_train = feature_train.iloc[:, :-1]
y_train = feature_train['score'].tolist()

In [17]:
# 7. TRAIN ALGORITHM
# Create a Random Forest Algorithm model
model_clf = RandomForestClassifier(n_estimators = 4, random_state = 42)
model_clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=4, random_state=42)

In [20]:
#SAVE MODEL
filename = '/content/drive/MyDrive/Dataset/Random_Forest_classifier_review.model'
pickle.dump(model_clf, open(filename, 'wb'))

# APPLY THE TRAINED LEARNER TO TEST NEW DATA
y_pred = model_clf.predict(x_test)


In [28]:
# CONFUSION MATRIX (CEK THE MDOEL PERFORMA)
lst_actual_class = y_test
lst_predicted_class = y_pred

# label ['POSITIF', 'NETRAL','NEGATIF'], [3, 2, 21]
lst_classes = [1, 2, 3]

# Compute multi-class confusion matrix
arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results
store_sens = [];
store_spec = [];
store_acc = [];
store_bal_acc = [];
store_prec = [];
store_fscore = [];
store_mcc = [];

# Loop for each taget label
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Predicted Performance of Digit Label/Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    precision = round(tp/(tp + fp), 3)
    fscore = round((2 * ((precision * sensitivity) / (precision + sensitivity))), 3)
    mcc = round((((tp * tn)-(fp * fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))),3)

    store_sens.append(sensitivity);
    store_spec.append(specificity);
    store_acc.append(accuracy);
    store_bal_acc.append(balanced_accuracy);
    store_prec.append(precision);
    store_fscore.append(fscore);
    store_mcc.append(mcc);
    
    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Sensitivity: {0}".format(sensitivity));
    print("Specificity: {0}".format(specificity));
    print("Accuracy: {0}".format(accuracy));
    print("Balanced Accuracy: {0}".format(balanced_accuracy));
    print("Precision: {0}".format(precision));
    print("F1-Score: {0}".format(fscore))
    print("MCC: {0}\n".format(mcc))

Predicted Performance of Digit Label/Class: 0
TP=1949, FP=207, TN=2664, FN=35
Sensitivity: 0.982
Specificity: 0.928
Accuracy: 0.95
Balanced Accuracy: 0.955
Precision: 0.904
F1-Score: 0.941
MCC: 0.901

Predicted Performance of Digit Label/Class: 1
TP=546, FP=32, TN=4048, FN=229
Sensitivity: 0.705
Specificity: 0.992
Accuracy: 0.946
Balanced Accuracy: 0.848
Precision: 0.945
F1-Score: 0.808
MCC: 0.788

Predicted Performance of Digit Label/Class: 2
TP=2002, FP=119, TN=2640, FN=94
Sensitivity: 0.955
Specificity: 0.957
Accuracy: 0.956
Balanced Accuracy: 0.956
Precision: 0.944
F1-Score: 0.949
MCC: 0.911



In [25]:
##########################################################
# 8. OVERALL - FINAL PREDICTION PERFORMANCE
##########################################################
print("Overall Performance Prediction:");
print("Sensitivity: {0}%".format(round(mean(store_sens)*100, 4)));
print("Specificity: {0}%".format(round(mean(store_spec)*100, 4)));
print("Accuracy: {0}%".format(round(mean(store_acc)*100, 4)));
print("Balanced Accuracy: {0}%".format(round(mean(store_bal_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(store_prec)*100, 4)));
print("F1-Score: {0}%".format(round(mean(store_fscore)*100, 4)))
print("MCC: {0}\n".format(round(mean(store_mcc), 4)))

Overall Performance Prediction:
Sensitivity: 92.5333%
Specificity: 97.4333%
Accuracy: 96.9%
Balanced Accuracy: 95.0%
Precision: 95.1667%
F1-Score: 93.6667%
MCC: 0.9143

